# Introduction

This competetion holds regional dialects on common subjects from 373 people representing diverse regional dialects across ten geographical areas like Rangpur, Kishoreganj, Narail, Chittagong, Narsingdi, Tangail, Barishal, Habiganj, Sylhet, and Sandwip. 

Here I have used tugstugi model for inference. The model includes:

## STT Model:

* OpenAI whisper-medium
* Huggingface trainer
* Trained on 8x 48GB RTX A6000
* bs=8 and lr=1e-5
* Train steps 50k
* Spectrogram dithering
* Spectrogram time and frequency masking
* Resampling 16khz->8khz->16khz as augmentation
* Inference with max_length=260, num_beams=4 and chunk_length_s=20.1s
* Libsonic based speed/pitch augmentation
* Datasets: OpenSLR 37, OpenSLR 53, MadASR, Shrutilipi, Macro, Kathbath, GoogleTTS generated audios and pseudo labeled YouTube videos

## Punctuation Model:

* AutoModelForTokenClassification google/muril-base-cased
* Huggingface trainer
* Labels: period, comma and question mark
* bs=64, lr=2e-4 and max_seq_length=512
* Ensemble of 4 models (using 6, 8, 11 and 12 layers of google/muril-base-cased)
* Normalized IndicCorp v2 Bangla dataset

model weight and inference notebook: https://www.kaggle.com/competitions/bengaliai-speech/discussion/447970

In [1]:
!cp /kaggle/input/bengali-eval-data/predict.py .

!cp -r ../input/python-packages2 ./
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/python-Levenshtein-0.12.2.tar.gz -f ./ --no-index
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index

cp: cannot stat '/kaggle/input/bengali-eval-data/predict.py': No such file or directory
jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/python-Levenshtein-0.12.2.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp310-cp310-linux_x86_64.whl size=79809 sha256=39fbcb21a9e05ceedd2068832039615136ea257aa2e45a37347551b768564888
  Stored in directory: /root/.cache/pip/wheels/f1/ab/b1/90d2068d73d15e52c1a65676d269a9f043b61221a29f7298e7
Successfully built python-Levenshtein
  Attempting uninstall: python-Levenshtein
    Found existing installation: python-Levenshtein 0.25.0
    Uninstalling python-Levenshtein-0.25.0:
      Successfully uninstalled python-Levenshtein-0.25.0
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl


In [2]:
import os
import csv
import time
import glob

MODEL = '/kaggle/input/bengali-ai-asr-submission/bengali-whisper-medium/'
PUNCT_MODELS = [
    '/kaggle/input/bengali-ai-asr-submission/punct-model-6layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-8layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-11layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-12layers/'
]
CHUNK_LENGTH_S = 20.1
ENABLE_BEAM = True


PUNCT_WEIGHTS = [[1.0, 1.4, 1.0, 0.8]]

if ENABLE_BEAM:
    BATCH_SIZE = 4
else:
    BATCH_SIZE = 8

    

DATASET_PATH = '/kaggle/input/yt-speech-chunks/chunks/'
    
import csv
import glob
import shutil
import librosa
import argparse
import warnings
from pathlib import Path
import transformers
print(transformers.__version__)
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

import warnings

warnings.filterwarnings("ignore")

files = list(glob.glob(DATASET_PATH + '/' + '*.wav'))
files += list(glob.glob(DATASET_PATH + '/' + '*.mp3'))
files.sort()

pipe = pipeline(task="automatic-speech-recognition",
                model=MODEL,
                tokenizer=MODEL,
                chunk_length_s=CHUNK_LENGTH_S, device=0, batch_size=BATCH_SIZE)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="bn", task="transcribe")

print("model loaded!")

4.38.2


2024-04-07 22:57:44.614244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 22:57:44.614348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 22:57:44.739647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model loaded!


In [3]:
def fix_repetition(text, max_count):
    uniq_word_counter = {}
    words = text.split()
    for word in text.split():
        if word not in uniq_word_counter:
            uniq_word_counter[word] = 1
        else:
            uniq_word_counter[word] += 1

    for word, count in uniq_word_counter.items():
        if count > max_count:
            words = [w for w in words if w != word]
    text = " ".join(words)
    return text

In [4]:
if ENABLE_BEAM:
    texts = pipe(files, generate_kwargs={"max_length": 260, "num_beams": 4})
else:
    texts = pipe(files)

In [5]:
preds = []
for i in texts:
    preds.append(i['text'])
len(preds)

542

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame({"id":files,"sentence":preds})

In [8]:
df['id'] = df['id'].apply(lambda x: x.replace(DATASET_PATH,""))
df.head()

,id,sentence
0,000000_000003.wav,অভ্যাসের বেলা আসুন নেই বালা দেওয়া বাবা দিন খান...
1,000000_000004.wav,খান্ডির আমোদ যে মানুষের সামনে আয় আঁকড়া তাই লজ্...
2,000000_000005.wav,আমার এই অবস্থা আছে না রে বাবা আমি আমার আল্লাহ ...
3,000000_000006.wav,আমার বাচ্চার দুয়াটা কিডনি নষ্ট
4,000000_000007.wav,এলাকি ভাঙচুর জারি পাইলাম তার কিছু থেকে আঁতর থে...


In [9]:
df.to_csv("submission.csv",index=False)